In [2]:
import pandas as pd

# 假设数据存储在 CSV 文件中
file_path = "./datasets/mapogu-4k-ce7-fd6.csv"  # 替换成你的文件路径

# 读取 CSV 文件
df = pd.read_csv(file_path)

# 初始化一个字典来计数每个 place_id 的出现次数
place_id_counter = {}

# 定义一个函数为每行生成 row_id
def generate_row_id(row):
    place_id = row['place_id']
    if place_id not in place_id_counter:
        place_id_counter[place_id] = 1
    else:
        place_id_counter[place_id] += 1
    return f"{place_id}_{place_id_counter[place_id]}"

# 创建新的 row_id 列
df['row_id'] = df.apply(generate_row_id, axis=1)


# 打印生成的 DataFrame
print(df)

# 保存回CSV（可选）
df.to_csv("./datasets/mapogu-4k-ce7-fd6_row_id.csv", index=False)


      category    place_id         place_name  comment_id  comment_user_id   
0          CE7   756993733  투썸플레이스 디지털미디어시티역점    10840268       2350725315  \
1          CE7   756993733  투썸플레이스 디지털미디어시티역점    10517631       1650546034   
2          CE7   756993733  투썸플레이스 디지털미디어시티역점    10349082       3371210705   
3          CE7   972496863      메가MGC커피 수색자이점     9461669       1231775476   
4          CE7   972496863      메가MGC커피 수색자이점     9246847       1030013027   
...        ...         ...                ...         ...              ...   
16553      FD6    15528437             광화문수제비     7025846        891589374   
16554      FD6  1195558165               리우키친    10188764        418219292   
16555      FD6  1195558165               리우키친    10149964       2377789139   
16556      FD6  1195558165               리우키친     7963312       2234012693   
16557      FD6  1594659846       만평우동 상암KGIT점    11038430        418219292   

                                         comment_content  comme

In [3]:
import pandas as pd

# 设置文件路径
file_path = './datasets/mapogu-4k-ce7-fd6_row_id.csv'

# 读取 CSV 文件
df = pd.read_csv(file_path)

# 获取行数
row_count = df.shape[0]


# 输出行数
print(f"文件中的行数为: {row_count}")

文件中的行数为: 16558


In [5]:
# 读取原始文件，并在开始和结尾添加方括号
input_file_path = './datasets/mapogu-4k-ce7-fd6_row_id_score.json'
output_file_path = './datasets/fixed_row_id_score.json'

try:
    # 读取原始文件并逐行写入，确保加入数组结构
    with open(input_file_path, 'r') as infile, open(output_file_path, 'w') as outfile:
        outfile.write('[')  # 在文件开始添加左方括号
        
        first_line = True  # 标记是否是第一行
        for line in infile:
            if not first_line:
                outfile.write(',\n')  # 在每行之间添加逗号分隔
            outfile.write(line.strip())
            first_line = False
        
        outfile.write(']')  # 在文件结尾添加右方括号
    
    print(f"文件修复完成，结果已写入 {output_file_path}")

except Exception as e:
    print(f"文件处理时发生错误: {e}")


文件修复完成，结果已写入 ./datasets/fixed_row_id_score.json


In [6]:
import json

try:
    # 读取 JSON 文件
    with open('./datasets/fixed_row_id_score.json', 'r') as file:
        data = json.load(file)
    
    # 筛选出符合条件和不符合条件的记录
    filtered_data = [item for item in data if -1 <= item['sentiment_score'] <= 1]
    removed_data = [item for item in data if not (-1 <= item['sentiment_score'] <= 1)]
    
    # 将筛选后的数据写回文件（或保存为新文件）
    with open('./datasets/filtered_row_id_score.json', 'w') as file:
        json.dump(filtered_data, file, indent=4)
    
    # 如果想保留被删除的记录
    if removed_data:
        with open('./datasets/removed_row_id_score.json', 'w') as file:
            json.dump(removed_data, file, indent=4)
    
    print(f"已删除 sentiment_score 不在 [-1, 1] 范围内的记录，剩余 {len(filtered_data)} 条。")
    if removed_data:
        print(f"不符合条件的记录已保存，数量为：{len(removed_data)}")

except FileNotFoundError:
    print("文件未找到，请检查路径是否正确。")
except json.JSONDecodeError:
    print("JSON 文件格式错误，请检查文件内容。")
except Exception as e:
    print(f"发生错误: {e}")


已删除 sentiment_score 不在 [-1, 1] 范围内的记录，剩余 13223 条。
不符合条件的记录已保存，数量为：613


In [7]:
import json
import csv

# 文件路径
json_file_path = './datasets/filtered_row_id_score.json'  # 你的JSON文件路径
csv_file_path = './datasets/mapogu-4k-ce7-fd6_row_id.csv'  # 你的CSV文件路径
output_csv_file_path = './datasets/updated_mapogu-4k-ce7-fd6_row_id.csv'  # 输出文件路径

# 读取JSON文件
with open(json_file_path, 'r') as json_file:
    json_data = json.load(json_file)

# 创建一个字典来映射row_id到sentiment_score
row_id_to_sentiment = {item['row_id']: item['sentiment_score'] for item in json_data}

# 读取CSV文件并逐行处理
with open(csv_file_path, 'r', encoding='utf-8') as csv_file, open(output_csv_file_path, 'w', newline='', encoding='utf-8') as output_csv_file:
    csv_reader = csv.reader(csv_file)
    csv_writer = csv.writer(output_csv_file)
    
    # 获取表头
    header = next(csv_reader)
    # 在表头中增加一个sentiment_score列
    header.append('sentiment_score')
    csv_writer.writerow(header)
    
    # 处理每一行数据
    for row in csv_reader:
        row_id = row[-1]  # 假设 row_id 在CSV文件的最后一列
        sentiment_score = row_id_to_sentiment.get(row_id, 'null')  # 如果没有找到匹配的row_id，sentiment_score为null
        row.append(sentiment_score)
        csv_writer.writerow(row)

print(f"处理完成，更新后的CSV文件已保存到 {output_csv_file_path}")


处理完成，更新后的CSV文件已保存到 ./datasets/updated_mapogu-4k-ce7-fd6_row_id.csv


In [10]:
import csv

# 文件路径
csv_file_path = './datasets/updated_mapogu-4k-ce7-fd6_row_id.csv'  # 你的现有CSV文件路径
output_csv_file_path = './datasets/final_mapogu-4k-ce7-fd6_row_id_scaled.csv'  # 最终输出的文件路径

# 定义缩放到 0-5 区间的函数
def scale_to_range(value, min_value, max_value, new_min, new_max):
    # 将 value 从 (min_value, max_value) 区间缩放到 (new_min, new_max) 区间
    return ((value - min_value) / (max_value - min_value)) * (new_max - new_min) + new_min

# 读取CSV文件并逐行处理
with open(csv_file_path, 'r', encoding='utf-8') as csv_file, open(output_csv_file_path, 'w', newline='', encoding='utf-8') as output_csv_file:
    csv_reader = csv.DictReader(csv_file)
    fieldnames = csv_reader.fieldnames + ['sentiment_comment_point_sum']  # 在表头中增加 sentiment_comment_point_sum 列
    csv_writer = csv.DictWriter(output_csv_file, fieldnames=fieldnames)
    
    # 写入表头
    csv_writer.writeheader()
    
    # 处理每一行数据
    for row in csv_reader:
        comment_point = float(row['comment_point'])  # 按名字获取 comment_point
        sentiment_score = row['sentiment_score']  # 按名字获取 sentiment_score

        if sentiment_score == 'null':  # 如果 sentiment_score 是 'null'
            sentiment_comment_point_sum = comment_point
        else:
            sentiment_score = float(sentiment_score)
            # 计算 sentiment_score 和 comment_point 的和
            total = sentiment_score + comment_point
            
            # 假设 sentiment_score 在 [-1, 1]，comment_point 在 [0, 5]
            # total 的可能范围是 [-1, 6]，我们将其缩放到 [0, 5] 区间
            sentiment_comment_point_sum = scale_to_range(total, -1, 6, 0, 5)

        # 保留两位小数
        sentiment_comment_point_sum = round(sentiment_comment_point_sum, 2)

        # 在行中添加 sentiment_comment_point_sum 列
        row['sentiment_comment_point_sum'] = sentiment_comment_point_sum
        csv_writer.writerow(row)

print(f"处理完成，结果已保存到 {output_csv_file_path}")


处理完成，结果已保存到 ./datasets/final_mapogu-4k-ce7-fd6_row_id_scaled.csv


In [11]:
import csv

# 文件路径
csv_file_path = './datasets/final_mapogu-4k-ce7-fd6_row_id_scaled.csv'  # 已经生成的CSV文件路径

# 计数器来跟踪是否有空值
empty_rows = []

# 读取CSV文件并逐行处理
with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    
    # 逐行检查 sentiment_comment_point_sum 列是否为空
    for row_num, row in enumerate(csv_reader, start=1):
        sentiment_comment_point_sum = row.get('sentiment_comment_point_sum', '').strip()
        
        # 检查是否为空值或为 'null'
        if sentiment_comment_point_sum == '' or sentiment_comment_point_sum == 'null':
            empty_rows.append(row_num)  # 记录空缺行的行号

# 输出结果
if empty_rows:
    print(f"发现 sentiment_comment_point_sum 列存在空缺值的行号: {empty_rows}")
else:
    print("所有行的 sentiment_comment_point_sum 列都有值。")


所有行的 sentiment_comment_point_sum 列都有值。
